In [1]:
import numpy as np
from math import *
import os
from os.path import join
import bpy
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipyvolume as ipv
from numpy.linalg import norm
import torch
import lie_learn.spaces.S2 as S2

ImportError: No module named bpy

In [ ]:
def plot_spheres(centers):
    fig = ipv.figure()
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)
    R = 2 * 1/3 * np.sqrt(3) * 1/2
    for center in centers:
        x = R * np.outer(np.cos(u), np.sin(v)) + center[0]
        y = R * np.outer(np.sin(u), np.sin(v))  + center[1]
        z = R * np.outer(np.ones(np.size(u)), np.cos(v)) + center[2]
        ipv.plot_mesh(x, y, z, u=y, v=x, wireframe=False, color="red")
    ipv.show()

In [20]:
def get_spheres(scale=1, subdiv=1, radius=0.562):
    middle_point_cache = {}
    
    def vertex(x, y, z):
        """ Return vertex coordinates fixed to the unit sphere """
        length = np.sqrt(x**2 + y**2 + z**2)
        return [(i * scale) / length for i in (x,y,z)]


    def middle_point(point_1, point_2):
        """ Find a middle point and project to the unit sphere """
        smaller_index = min(point_1, point_2)
        greater_index = max(point_1, point_2)

        key = "{0}-{1}".format(smaller_index, greater_index)

        if key in middle_point_cache:
            return middle_point_cache[key]

        # If it's not in cache, then we can cut it
        vert_1 = verts[point_1]
        vert_2 = verts[point_2]
        middle = [sum(i)/2 for i in zip(vert_1, vert_2)]

        verts.append(vertex(*middle))

        index = len(verts) - 1
        middle_point_cache[key] = index

        return index

    PHI = (1 + np.sqrt(5)) / 2

    verts = [ vertex(-1, PHI, 0), 
         vertex( 1, PHI, 0), 
         vertex(-1, -PHI, 0), 
         vertex( 1, -PHI, 0), 
         vertex(0, -1, PHI), 
         vertex(0, 1, PHI), 
         vertex(0, -1, -PHI), 
         vertex(0, 1, -PHI), 
         vertex( PHI, 0, -1), 
         vertex( PHI, 0, 1), 
         vertex(-PHI, 0, -1), 
         vertex(-PHI, 0, 1), ]

    faces = [ 
        # 5 faces around point 0 
        [0, 11, 5], 
        [0, 5, 1], 
        [0, 1, 7], 
        [0, 7, 10], 
        [0, 10, 11], 
        # Adjacent faces 
        [1, 5, 9], 
        [5, 11, 4], 
        [11, 10, 2], 
        [10, 7, 6], 
        [7, 1, 8], 
        # 5 faces around 3 
        [3, 9, 4], 
        [3, 4, 2], 
        [3, 2, 6], 
        [3, 6, 8], 
        [3, 8, 9], 
        # Adjacent faces 
        [4, 9, 5], 
        [2, 4, 11], 
        [6, 2, 10], 
        [8, 6, 7], 
        [9, 8, 1], 
    ]
    
#     # Generate based dots on Sphere surface [12 spheres -> 42 spheres]
#     for i in range(subdiv): 
#         faces_subdiv = [] 
#         for tri in faces: 
#             v1 = middle_point(tri[0], tri[1]) 
#             v2 = middle_point(tri[1], tri[2]) 
#             v3 = middle_point(tri[2], tri[0]) 
#             faces_subdiv.append([tri[0], v1, v3]) 
#             faces_subdiv.append([tri[1], v2, v1]) 
#             faces_subdiv.append([tri[2], v3, v2]) 
#             faces_subdiv.append([v1, v2, v3]) 
#         faces = faces_subdiv

    def get_radius(scale, verts):
        min_dist = find_min_dist(verts)
        d = 0.5 * min_dist / np.cos(np.pi / 6) 

        x = verts[5]

        v = np.random.rand(3)
        v = v - np.matmul(v, x) * x
        v = v / norm(v) * d

        p = np.cos(norm(v)) * x + np.sin(norm(v)) * v / norm(v)

        origin = np.array([0, 0, 0])
        center = 0.5 * (origin + x)

        radius = euclidean_distance(p, center)

        return radius
    
    # Get embedding sphere centers
    centers = []
    origin = np.array([0, 0, 0])
    for vert in verts:
        center = 0.5 * (origin + vert)
        centers.append(center)
    
    return centers, radius

In [21]:
centers, radius = get_spheres()

In [22]:
plot_spheres(centers)

In [23]:
len(centers)

12